In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import gymnasium as gym
from collections import Counter

In [2]:
class DeepQNetwork(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DeepQNetwork, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [34]:
dqn = DeepQNetwork(4, 2)
dqn.load_state_dict(torch.load("../models/gorila.pth"))
# dqn.load_state_dict(torch.load("../checkpoints/checkpoint-190000.pth"))

<All keys matched successfully>

In [36]:
dqn.eval()
env = gym.make('CartPole-v1', render_mode="human")
state, _ = env.reset()
actions = []
rewards = 0

for _ in range(1_000):
    q_values = dqn(torch.tensor(np.array(state)).unsqueeze(0))
    action = q_values.argmax().item()
    actions.append(action)
    # print(action, q_values)
    state, reward, terminated, truncated, _ = env.step(action)
    rewards += reward
    if terminated or truncated:
        break

print("Total reward:", rewards)
print(Counter(actions))
env.close()

Total reward: 500.0
Counter({1: 250, 0: 250})
